Task 1:
Here I am cleaning the orignal data and making a new csv to play with

In [1]:
# Libraries imported to build queries
import pandas as pd
import sqlite3
from sqlalchemy import create_engine

#Establish a coonnection with a database
con = sqlite3.connect('original_db.db')
# Creates a object to retrieve resultss
cur = con.cursor()

# Read the first line of the CSV file to extract the header
with open('no_town.csv', 'r') as file:
    header_line = file.readline().strip()

# Remove quotes from the header
header_line = header_line.replace("'", "")

# Read the CSV file into a DataFrame, specifying the modified header
df = pd.read_csv('no_town.csv', delimiter=',', header=None, skiprows=1)

# Set the modified header to the DataFrame
df.columns = header_line.split(',')

# Remove quotes from all words in the DataFrame
df = df.applymap(lambda x: x.replace("'", "") if isinstance(x, str) else x)

# Change the second occurance of name
columns = df.columns.tolist()
seen = set()
for i, col in enumerate(columns):
    if col in seen:
        columns[i] = 'title'
    seen.add(col)
df.columns = columns
df.columns = [col.strip(' ') for col in df.columns]

# Save the cleaned DataFrame back to a CSV file
df.to_csv('cleaned_csv_file.csv', index=False)


Task 2

In [2]:
# Table for Musicians
cur.execute("""
  CREATE TABLE IF NOT EXISTS Musicians (
      name TEXT NOT NULL,
      number INTEGER NOT NULL,
      street_name TEXT NOT NULL,
      street_type TEXT NOT NULL,
      ssn TEXT PRIMARY KEY
  )
""")

# Table for Instruments
cur.execute("""
  CREATE TABLE IF NOT EXISTS Instruments (
    instrument_id INTEGER PRIMARY KEY,
    instrument_type TEXT,
    key TEXT
)
""")

# Table Albums
cur.execute("""
  CREATE TABLE IF NOT EXISTS Albums (
    album_id INTEGER PRIMARY KEY,
    title TEXT,
    copyright_date DATE,
    album_type TEXT
)
""")


In [3]:
# album_musicians table for task 4
cur.execute('''
  CREATE TABLE album_musicians (
    album_id TEXT,
    ssn TEXT,
    FOREIGN KEY(album_id) REFERENCES albums(album_id),
    FOREIGN KEY(ssn) REFERENCES musicians(ssn),
    PRIMARY KEY(album_id, ssn)
  )''')



Task 3

In [4]:
# Create SQLite database engine
engine = create_engine('sqlite:///new_db.db')

# Read data from CSV into a DataFrame
data_df = pd.read_csv('cleaned_csv_file.csv')

# Load each dataframe with desired values according to sql tables

# Check if the DataFrame is empty
if data_df.empty:
    print("DataFrame is empty. Cannot extract columns.")
else:
    # Verify the actual column names in the DataFrame
    actual_column_names = data_df.columns
    if not all(col in actual_column_names for col in ['name', 'num', 'street', 'street_type', 'ssn']):
        print("Column names do not match the DataFrame.")
    else:
        # Extract the desired columns
        musicians_df = data_df[['name', 'num', 'street', 'street_type', 'ssn']]

# print(musicians_df)

# Check if the DataFrame is empty
if data_df.empty:
    print("DataFrame is empty. Cannot extract columns.")
else:
    # Verify the actual column names in the DataFrame
    actual_column_names = data_df.columns
    if not all(col in actual_column_names for col in ['instrument_id', 'instrument_type', 'key']):
        print("Column names do not match the DataFrame.")
    else:
        # Extract the desired columns
        instruments_df = data_df[['instrument_id', 'instrument_type', 'key']]

# print(instruments_df)

# Check if the DataFrame is empty
if data_df.empty:
    print("DataFrame is empty. Cannot extract columns.")
else:
    # Verify the actual column names in the DataFrame
    actual_column_names = data_df.columns
    if not all(col in actual_column_names for col in ['album_id', 'title', 'date', 'album_type']):
        print("Column names do not match the DataFrame.")
    else:
        # Extract the desired columns
        albums_df = data_df[['album_id', 'title', 'date', 'album_type']]

# print(albums_df)

# Dataframe for task 4
# Check if the DataFrame is empty
if data_df.empty:
    print("DataFrame is empty. Cannot extract columns.")
else:
    # Verify the actual column names in the DataFrame
    actual_column_names = data_df.columns
    if not all(col in actual_column_names for col in ['album_id', 'ssn']):
        print("Column names do not match the DataFrame.")
    else:
        # Extract the desired columns
        album_musicians_df = data_df[['album_id', 'ssn',]]


# print(album_musicians_df)


Task 4: Summary

In [5]:
# 1: Total number of musicians and list of musicians (name and ssn)
total_musicians = len(musicians_df)

# 2: Total number of albums and list of albums recorded at Notown (name and album id)
total_albums = len(albums_df)

# 3: Total number of instruments and list of instruments at Notown (name, key, and id)
total_instruments = len(instruments_df)

# 4: Table consisting of the names of musicians and the total number of albums written by them

# Group album_musicians_df by SSN and count the number of albums for each musician
albums_per_musician = album_musicians_df.groupby('ssn').size().reset_index(name='total_albums')

# Merge with musicians_df to include musician names
task4_df = pd.merge(musicians_df, albums_per_musician, on='ssn', how='left')

# account for musicians with no albums
task4_df['total_albums'].fillna(0, inplace=True)

# Remove duplicates
task4_df.drop_duplicates(inplace=True)


# Print the results
print("Task 1:")
print("Total number of musicians:", total_musicians)
print("List of musicians:")
print(musicians_df[['name', 'ssn']])

print("\nTask 2:")
print("Total number of albums:", total_albums)
print("List of albums:")
print(albums_df[['title', 'album_id']])

print("\nTask 3:")
print("Total number of instruments:", total_instruments)
print("List of instruments:")
print(instruments_df[['instrument_id', 'key', 'instrument_type']])

print("\nTask 4:")
print("Table of musicians and total albums written by them:")
print(task4_df[['name', 'total_albums']])

print("\nTask 4.4:")
print("Table of per musician count. It will return a table consists of musician info count where musician info should be all the attributes you designed on task 2 to represent a musician")
print(task4_df)


Task 1:
Total number of musicians: 370
List of musicians:
         name        ssn
0     Antonio  100000000
1     Antonio  100000000
2     Antonio  100000000
3     Antonio  100000000
4     Antonio  100000000
..        ...        ...
365    Ariana  100000043
366    Ariana  100000043
367    Ariana  100000043
368    Ariana  100000043
369    Ariana  100000043

[370 rows x 2 columns]

Task 2:
Total number of albums: 370
List of albums:
    title  album_id
0       F         7
1       X        21
2       X        21
3       X        21
4       V        24
..    ...       ...
365     J        10
366     J        10
367     X        21
368     X        21
369     X        21

[370 rows x 2 columns]

Task 3:
Total number of instruments: 370
List of instruments:
     instrument_id      key instrument_type
0                9        C           flute
1                5        B     synthesizer
2                8   B-flat     synthesizer
3                9        C           flute
4                6

In [6]:
con.commit()
con.close()